In [47]:
# Carica il file CSV con tabulazione come separatore
df = pd.read_csv("./filtered_delivery_data.csv")

# Normalizza i nomi delle colonne
df.columns = df.columns.str.strip().str.upper()

# Funzione per aggiornare VEHICLETYPE in base a SERVICETYPE
def update_vehicletype(row):
    if pd.isna(row['VEHICLETYPE']) or row['VEHICLETYPE'] not in ['express', 'standard']:
        if 'corriere espresso' in str(row['SERVICETYPE']).lower():
            return 'express'
        else:
            return 'standard'
    return row['VEHICLETYPE']

# Applica la funzione sulla colonna 'VEHICLETYPE'
df['VEHICLETYPE'] = df.apply(update_vehicletype, axis=1)

# Salva il file aggiornato
df.to_csv("./filtered_delivery_data.csv", index=False)

print("File aggiornato salvato come 'filtered_delivery_data.csv'")

# Trova i tipi unici e il loro conteggio
unique_vehicle_types = df['VEHICLETYPE'].nunique()
all_vehicle_types = df['VEHICLETYPE'].unique()

print(f"Numero di tipi diversi di veicoli: {unique_vehicle_types}")
print(f"Tipi di veicoli presenti: {all_vehicle_types}")


File aggiornato salvato come 'filtered_delivery_data.csv'
Numero di tipi diversi di veicoli: 2
Tipi di veicoli presenti: ['express' 'standard']


In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Caricare il dataset
df = pd.read_csv("./delivery_data.csv")

# Selezionare le caratteristiche numeriche per la predizione
features = ['GROSS_WEIGHT_KG', 'NET_WEIGHT_KG', 'VOLUME_M3', 'DECLARED_DISTANCE_KM', 'DELIVERY_TIME_HH']

def predict_missing_values(df, target_column):
    print(f"\n{'='*50}")
    print(f"Analisi predittiva per: {target_column}")
    print('='*50)
    
    df_known = df.dropna(subset=[target_column])
    df_unknown = df[df[target_column].isna()]
    
    if df_known.empty or df_unknown.empty:
        print(f"Nessun valore da predire per {target_column}")
        return df
    
    X = df_known[features]
    y = df_known[target_column].astype('category').cat.codes
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Valutazione del modello
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\nMetriche di accuratezza per {target_column}:")
    print(f"Accuratezza globale: {accuracy:.4f}")
    
    print("\nReport dettagliato di classificazione:")
    print(classification_report(y_test, y_pred))
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nImportanza delle features:")
    for idx, row in feature_importance.iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")
    
    # Predizione dei valori mancanti
    df_unknown[target_column] = model.predict(df_unknown[features])
    df = pd.concat([df_known, df_unknown])
    
    print(f"\nNumero di valori predetti: {len(df_unknown)}")
    
    return df

# Eseguire le predizioni
print("Iniziando l'analisi predittiva...")
df = predict_missing_values(df, 'VEHICLETYPE')
df = predict_missing_values(df, 'SERVICETYPE')
print("\nRegressione completata con successo!")

Iniziando l'analisi predittiva...

Analisi predittiva per: VEHICLETYPE

Metriche di accuratezza per VEHICLETYPE:
Accuratezza globale: 0.9002

Report dettagliato di classificazione:
              precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       0.94      1.00      0.97        15
           2       0.89      0.81      0.85       785
           3       0.00      0.00      0.00        20
           4       0.25      0.08      0.12        26
           5       0.87      0.93      0.90       368
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         1
           8       0.92      0.96      0.94      1644

    accuracy                           0.90      2867
   macro avg       0.54      0.44      0.46      2867
weighted avg       0.89      0.90      0.89      2867


Importanza delle features:
DECLARED_DISTANCE_KM: 0.3417
DELIVERY_TIME_HH: 0.2357
VOLUME_M3: 0.1450
NET_WEIGHT_KG: